In [23]:
import geopandas as gpd
import networkx as nx
from libpysal import weights
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [145]:
data_dir = Path("../data")

# read points of interest
zion_points = gpd.read_file(data_dir / "zion_points.gpkg")
zion_points.head()

In [ ]:
# map the points
fig, ax = plt.subplots()
zion_points.plot(ax=ax, color='black')

for x, y, label in zip(zion_points.geometry.x, zion_points.geometry.y, zion_points.index):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

In [127]:
coordinates = np.column_stack((zion_points.geometry.x, zion_points.geometry.y))

n_thresholds = 5
fig, axes = plt.subplots(1, n_thresholds, figsize=(16, 3))
for i, threshold in enumerate(np.linspace(0.01, 0.07, n_thresholds)):

    dist = weights.DistanceBand.from_array(coordinates, threshold=threshold)
    dist_graph = dist.to_networkx()
    positions = dict(zip(dist_graph.nodes, coordinates))

    nx.draw(dist_graph, pos=positions, node_size=10, ax=axes[i])
    axes[i].set_title(f"Threshold: {threshold:.3f}")


In [143]:
fig, ax = plt.subplots()

nx.draw_networkx_edges(dist_graph, positions, edge_color='grey', width=0.5, ax=ax)

start = 9
end = 14
shortest_path = nx.algorithms.shortest_paths.generic.shortest_path(dist_graph, start, end)
path_edges = list(zip(shortest_path, shortest_path[1:]))
nx.draw_networkx_nodes(dist_graph, positions, nodelist=shortest_path, node_color='g', node_size=15, ax=ax)
nx.draw_networkx_edges(dist_graph, positions, edgelist=path_edges, edge_color='g', ax=ax)